In [1]:
import numpy as np
import sympy as sp
import copy

In [2]:
s11, s22, s33, s12, s23, s13 = sp.symbols('\sigma_11,\sigma_22,\sigma_33,\sigma_12,\sigma_23,\sigma_13')
H = 7
G = 4
F = 2
N = 1
L = 7
M = 2
hill_yield = sp.sqrt((F*(s22 - s33)**2 + G*(s33 - s11)**2 + H*(s11 - s22)**2) + 2*L*s23**2 + 2*M*s13**2 + 2*N*s12**2)
hill_yield

sqrt(2*\sigma_12**2 + 4*\sigma_13**2 + 14*\sigma_23**2 + 4*(-\sigma_11 + \sigma_33)**2 + 7*(\sigma_11 - \sigma_22)**2 + 2*(\sigma_22 - \sigma_33)**2)

In [3]:
def test_hill_YS(stress_vector):
    s11 = stress_vector[0]
    s22 = stress_vector[1]
    s33 = stress_vector[2]
    s23 = stress_vector[3]
    s13 = stress_vector[4]
    s12 = stress_vector[5]
    hill_yield = np.sqrt((F*(s22 - s33)**2 + G*(s33 - s11)**2 + H*(s11 - s22)**2)/2 + L*s23**2 + M*s13**2 + N*s12**2)
    return hill_yield

In [4]:
index_2_stress_value = { 0: s11, 1: s22, 2: s33, 3: s23, 4: s13, 5: s12 }

stress_state_collection = []

number_of_eqps = 6

transpose_format =  [ [] for i in range(number_of_eqps) ]

for EE, eqps in enumerate(np.linspace(0, 0.05, number_of_eqps)):
    yield_stress_nominal = 10 + 990*eqps
    # 6x6 cominbations, let's add ONE point per stress combo
    for i in range(6):
        for j in range(6):
            if i == j:
                continue
            stress_vec_1 = np.zeros(7)
            stress_vec_1[6] = eqps
            stress_vec_2 = np.zeros(7)
            stress_vec_2[6] = eqps
            # Set the main stress (i) as the yield stress nominal
            hill = copy.copy(hill_yield) - yield_stress_nominal
            major_i = index_2_stress_value[i]
            minor_j = index_2_stress_value[j]
            for k in range(6):
                #print('k comp', index_2_stress_value[k], [major_i, minor_j])
                if index_2_stress_value[k] not in [major_i, minor_j]:
                    aux = index_2_stress_value[k]
                    #print(aux)
                    hill = hill.subs({aux: 0})
            hill = hill.subs({major_i: yield_stress_nominal/5.})
            #print(hill)
            stress_adj = sp.solve(hill, minor_j)
            if len(stress_adj) == 1:
                stress_vec_1[i] = yield_stress_nominal/5.
                stress_vec_1[j] = stress_adj[0]
                stress_state_collection.append(stress_vec_1)
                transpose_format[EE].append(stress_vec_1)
            else:
                stress_vec_1[i] = yield_stress_nominal/5.
                stress_vec_2[i] = yield_stress_nominal/5.
                stress_vec_1[j] = stress_adj[0]
                stress_vec_2[j] = stress_adj[1]
                stress_state_collection.append(stress_vec_1)
                stress_state_collection.append(stress_vec_2)
                transpose_format[EE].append(stress_vec_1)
                transpose_format[EE].append(stress_vec_2)

transpose_format = np.array(transpose_format)
#stress_state_collection, 
np.all(np.isreal(stress_state_collection))

TypeError: Cannot convert complex to float

In [ ]:
P = np.array( [ [ G+H, -H, -G, 0, 0, 0 ],
                [-H, F+H, -F, 0,0,0],
                 [-G, -F, F+G, 0, 0, 0],
                  [0,0,0, 2*L, 0, 0],
                   [0,0,0,0, 2*M, 0],
                    [0,0,0,0,0,2*N] ])
for ss in stress_state_collection:
    #assert(np.isclose( test_hill_YS(ss), 10 + 990*ss[-1]))
    #print(test_hill_YS(ss))
    eqps = ss[-1]
    print(ss[0:6] @ P*1/(10 + 990*eqps)**2 @ ss[0:6])

1.0
1.0000000000000002
1.0
1.0
0.9999999999999988
0.9999999999999988
0.9999999999999989
0.9999999999999989
0.9999999999999992
0.9999999999999992
0.9999999999999998
1.0000000000000002
1.0
1.0
0.9999999999999988
0.9999999999999988
0.9999999999999988
0.9999999999999988
0.9999999999999992
0.9999999999999992
0.999999999999999
0.999999999999999
0.999999999999999
0.999999999999999
0.9999999999999969
0.9999999999999969
0.9999999999999969
0.9999999999999969
0.9999999999999971
0.9999999999999971
1.0000000000000013
1.0000000000000013
1.0000000000000013
1.0000000000000013
1.0000000000000002
1.0000000000000002
0.9999999999999999
0.9999999999999999
1.0000000000000002
1.0000000000000002
1.0000000000000013
1.0000000000000013
1.0000000000000013
1.0000000000000013
1.0000000000000002
1.0000000000000002
0.9999999999999999
0.9999999999999999
1.0000000000000002
1.0000000000000002
0.999999999999999
0.999999999999999
0.999999999999999
0.999999999999999
1.000000000000007
1.000000000000007
0.9999999999999931
0.

In [ ]:
bingo_format = []
# Bingo format is the direct stress trajectores
number_of_ss_curves = transpose_format.shape[1]
print(number_of_ss_curves)
for ss_curve in range(number_of_ss_curves):
    # Grab the ss curve trajectories
    trajectory = transpose_format[:, ss_curve, :]
    bingo_format.extend(trajectory)
    bingo_format.append([ np.nan for _ in range(7) ])
bingo_format = np.array(bingo_format)
bingo_format = bingo_format[:-1]
np.savetxt('hill48_bingo.csv', bingo_format, delimiter=" ")

60


In [ ]:
bingo_transpose_format = []
for _ in range(number_of_eqps):
    bingo_transpose_format.extend(transpose_format[_, :, :])
    bingo_transpose_format.append([ np.nan for _ in range(7) ])
bingo_transpose_format = np.array(bingo_transpose_format)
bingo_transpose_format = bingo_transpose_format[:-1]
np.savetxt('hill48_bingo_transpose.csv', bingo_transpose_format, delimiter=" ")